In [2]:
from collections import deque


DIRECT = True
REVERSE = False
# print(full_msg)

def label_content_bound(label: str, content: str, margin = 2):
    return max(len(label), len(content)) + margin


def label_and_content(label: str, content, filling: str = ' ') -> tuple[str, str]:
    printed_content = filling.join(str(bit) for bit in content)
    bound = label_content_bound(label, printed_content)
    label = label.center(bound)
    printed_content = printed_content.center(bound)
    return label, printed_content


def print_labels_and_content(full_msg: deque, state: deque, out_res: list) -> None:
    signal_label, printed_signal = label_and_content('Сигнал:', reversed(full_msg))
    state_label, printed_state = label_and_content('Регистры:', state, '|-|')
    out_label, printed_out = label_and_content('Out:', out_res[::-1])

    print(signal_label + state_label + out_label)
    print(printed_signal + printed_state + printed_out)
    print('')


def register_multiplication(n: int, q: int, begin_state: list, out_poly: list, msg: list):
    state = deque(begin_state)
    full_msg = deque(msg + [0] * n)
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, out_res)
    
    for i in range(1, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        leftovers.append(state.pop())
        to_out = sum([look * pos for look, pos in zip(out_poly, state)]) % q
        out_res.append(to_out)
        print(f'Шаг {i}:')
        print_labels_and_content(full_msg, state, out_res)

In [3]:
n = 3
q = 2
begin_state = [0, 0, 0]
out_poly = [1, 0, 1]
msg = [1, 1, 1]


register_multiplication(n, q, begin_state, out_poly, msg)

Начальное состояние:
   Сигнал:    Регистры:  Out: 
 0 0 0 1 1 1  0|-|0|-|0       

Шаг 1:
  Сигнал:   Регистры:  Out: 
 0 0 0 1 1  1|-|0|-|0   1   

Шаг 2:
 Сигнал:  Регистры:  Out: 
 0 0 0 1  1|-|1|-|0  1 1  

Шаг 3:
 Сигнал:  Регистры:   Out: 
  0 0 0   1|-|1|-|1  0 1 1 

Шаг 4:
 Сигнал:  Регистры:    Out:  
   0 0    0|-|1|-|1  1 0 1 1 

Шаг 5:
 Сигнал:  Регистры:     Out:   
    0     0|-|0|-|1  1 1 0 1 1 



In [4]:
def check_length(left, right, label: str):
    if len(left) != len(right):
        raise ValueError(f'Args of different length in {label}: {left} and {right}')


def GF_mul(left, right, q: int):
    check_length(left, right, 'GF_mul')
    return sum([l * r for l, r in zip(left, right)]) % q


def GF_sum(left: int, right: int, q: int):
    return (left + right) % q


def register_division(n: int, q: int, begin_state: list[int], out_poly: list[int], den: list[int], msg: list[int], max_len: int = 10):
    len_den = len(den)
    len_begin = len(begin_state)
    
    if len_den - 1 > len_begin:
        state = deque(begin_state + [0] * (len(den) - len(begin_state) - 1))
    elif len_begin + 1 > len_den:
        den = den.copy()
        den = den + [0] * (len_begin - len_den + 1)
        state = deque(begin_state)
    else:
        state = deque(begin_state)
    full_msg = deque(msg + [0] * max_len)
    len_state = len(state)
    len_out_poly = len(out_poly)
    if len_state > len_out_poly:
        out_poly = out_poly.copy()
        out_poly = out_poly + [0] * (len_state - len_out_poly)
    elif len_state < len_out_poly:
        state = state + [0] * (len_out_poly - len_state)
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    target = [i for i, x in enumerate(den) if x][0]
    defer = [i for i, x in enumerate(out_poly) if x][0]
    den = den.copy()
    den[target] = 0
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, out_res)
    print(defer)
    for i in range(0, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        state[target] = GF_sum(state[target], GF_mul(den, state, q), q)
        leftover = state.pop()
        leftovers.append(leftover)
        if i >= defer:
            to_out = GF_mul(out_poly, state, q)
            out_res.append(to_out)
        print(f'Шаг {i + 1}:')
        print_labels_and_content(full_msg, state, out_res)

In [13]:
n = 3
q = 2
begin_state = [0]
numerator = [1]
denominator = [1, 1]
msg = [1]

register_division(n, q, begin_state, numerator, denominator, msg, 5)

Начальное состояние:
   Сигнал:    Регистры:  Out: 
 0 0 0 0 0 1      0           

0
Шаг 1:
  Сигнал:   Регистры:  Out: 
 0 0 0 0 0      1       1   

Шаг 2:
 Сигнал:  Регистры:  Out: 
 0 0 0 0      1      1 1  

Шаг 3:
 Сигнал:  Регистры:   Out: 
  0 0 0       1      1 1 1 

Шаг 4:
 Сигнал:  Регистры:    Out:  
   0 0        1      1 1 1 1 

Шаг 5:
 Сигнал:  Регистры:     Out:   
    0         1      1 1 1 1 1 

Шаг 6:
 Сигнал:  Регистры:      Out:    
              1      1 1 1 1 1 1 



## Примеры

### Умножение полиномов

In [6]:
n = 7
q = 2
begin_state = [0] * n
out_poly = [1] * n
msg = [1, 1]


register_multiplication(n, q, begin_state, out_poly, msg)

Начальное состояние:
      Сигнал:               Регистры:          Out: 
 0 0 0 0 0 0 0 1 1  0|-|0|-|0|-|0|-|0|-|0|-|0       

Шаг 1:
     Сигнал:              Регистры:          Out: 
 0 0 0 0 0 0 0 1  1|-|0|-|0|-|0|-|0|-|0|-|0   1   

Шаг 2:
    Сигнал:             Регистры:          Out: 
 0 0 0 0 0 0 0  1|-|1|-|0|-|0|-|0|-|0|-|0  0 1  

Шаг 3:
   Сигнал:            Регистры:           Out: 
 0 0 0 0 0 0  0|-|1|-|1|-|0|-|0|-|0|-|0  0 0 1 

Шаг 4:
  Сигнал:           Регистры:            Out:  
 0 0 0 0 0  0|-|0|-|1|-|1|-|0|-|0|-|0  0 0 0 1 

Шаг 5:
 Сигнал:          Регистры:             Out:   
 0 0 0 0  0|-|0|-|0|-|1|-|1|-|0|-|0  0 0 0 0 1 

Шаг 6:
 Сигнал:          Регистры:              Out:    
  0 0 0   0|-|0|-|0|-|0|-|1|-|1|-|0  0 0 0 0 0 1 

Шаг 7:
 Сигнал:          Регистры:               Out:     
   0 0    0|-|0|-|0|-|0|-|0|-|1|-|1  0 0 0 0 0 0 1 

Шаг 8:
 Сигнал:          Регистры:                Out:      
    0     0|-|0|-|0|-|0|-|0|-|0|-|1  1 0 0 0 0 0 0 1 



In [7]:
n = 4
q = 2
begin_state = [0] * n
out_poly = [1, 0, 1, 1]
msg = [1, 0, 1, 1, 1]


register_multiplication(n, q, begin_state, out_poly, msg)

Начальное состояние:
      Сигнал:         Регистры:    Out: 
 0 0 0 0 1 1 1 0 1  0|-|0|-|0|-|0       

Шаг 1:
     Сигнал:        Регистры:    Out: 
 0 0 0 0 1 1 1 0  1|-|0|-|0|-|0   1   

Шаг 2:
    Сигнал:       Регистры:    Out: 
 0 0 0 0 1 1 1  0|-|1|-|0|-|0  0 1  

Шаг 3:
   Сигнал:      Регистры:     Out: 
 0 0 0 0 1 1  1|-|0|-|1|-|0  0 0 1 

Шаг 4:
  Сигнал:     Регистры:      Out:  
 0 0 0 0 1  1|-|1|-|0|-|1  0 0 0 1 

Шаг 5:
 Сигнал:    Регистры:       Out:   
 0 0 0 0  1|-|1|-|1|-|0  0 0 0 0 1 

Шаг 6:
 Сигнал:    Регистры:        Out:    
  0 0 0   0|-|1|-|1|-|1  0 0 0 0 0 1 

Шаг 7:
 Сигнал:    Регистры:         Out:     
   0 0    0|-|0|-|1|-|1  0 0 0 0 0 0 1 

Шаг 8:
 Сигнал:    Регистры:          Out:      
    0     0|-|0|-|0|-|1  1 0 0 0 0 0 0 1 



In [8]:
n = 4
q = 2
begin_state = [0] * n
out_poly = [1, 1, 0, 1]
msg = [1, 1, 1, 0, 1]


register_multiplication(n, q, begin_state, out_poly, msg)

Начальное состояние:
      Сигнал:         Регистры:    Out: 
 0 0 0 0 1 0 1 1 1  0|-|0|-|0|-|0       

Шаг 1:
     Сигнал:        Регистры:    Out: 
 0 0 0 0 1 0 1 1  1|-|0|-|0|-|0   1   

Шаг 2:
    Сигнал:       Регистры:    Out: 
 0 0 0 0 1 0 1  1|-|1|-|0|-|0  0 1  

Шаг 3:
   Сигнал:      Регистры:     Out: 
 0 0 0 0 1 0  1|-|1|-|1|-|0  0 0 1 

Шаг 4:
  Сигнал:     Регистры:      Out:  
 0 0 0 0 1  0|-|1|-|1|-|1  0 0 0 1 

Шаг 5:
 Сигнал:    Регистры:       Out:   
 0 0 0 0  1|-|0|-|1|-|1  0 0 0 0 1 

Шаг 6:
 Сигнал:    Регистры:        Out:    
  0 0 0   0|-|1|-|0|-|1  0 0 0 0 0 1 

Шаг 7:
 Сигнал:    Регистры:         Out:     
   0 0    0|-|0|-|1|-|0  0 0 0 0 0 0 1 

Шаг 8:
 Сигнал:    Регистры:          Out:      
    0     0|-|0|-|0|-|1  1 0 0 0 0 0 0 1 



#### Деление полиномов

In [9]:
n = 8
q = 2
begin_state = [0] * n
numerator = [1, 0, 0, 0, 0, 0, 0, 1]
denominator = [1, 1]
msg = [1]

register_division(n, q, begin_state, numerator, denominator, msg, n - 1)

Начальное состояние:
     Сигнал:                Регистры:            Out: 
 0 0 0 0 0 0 0 1  0|-|0|-|0|-|0|-|0|-|0|-|0|-|0       

0
Шаг 1:
    Сигнал:               Регистры:            Out: 
 0 0 0 0 0 0 0  1|-|0|-|0|-|0|-|0|-|0|-|0|-|0   1   

Шаг 2:
   Сигнал:              Регистры:            Out: 
 0 0 0 0 0 0  1|-|1|-|0|-|0|-|0|-|0|-|0|-|0  1 1  

Шаг 3:
  Сигнал:             Регистры:             Out: 
 0 0 0 0 0  1|-|1|-|1|-|0|-|0|-|0|-|0|-|0  1 1 1 

Шаг 4:
 Сигнал:            Регистры:              Out:  
 0 0 0 0  1|-|1|-|1|-|1|-|0|-|0|-|0|-|0  1 1 1 1 

Шаг 5:
 Сигнал:            Регистры:               Out:   
  0 0 0   1|-|1|-|1|-|1|-|1|-|0|-|0|-|0  1 1 1 1 1 

Шаг 6:
 Сигнал:            Регистры:                Out:    
   0 0    1|-|1|-|1|-|1|-|1|-|1|-|0|-|0  1 1 1 1 1 1 

Шаг 7:
 Сигнал:            Регистры:                 Out:     
    0     1|-|1|-|1|-|1|-|1|-|1|-|1|-|0  1 1 1 1 1 1 1 

Шаг 8:
 Сигнал:            Регистры:                  Out:      
          1|-

In [10]:
n = 8
q = 2
begin_state = [0] * n
numerator = [1, 0, 0, 0, 0, 0, 0, 1]
denominator = [1, 1, 0, 1]
msg = [1]

register_division(n, q, begin_state, numerator, denominator, msg, n - 1)

Начальное состояние:
     Сигнал:                Регистры:            Out: 
 0 0 0 0 0 0 0 1  0|-|0|-|0|-|0|-|0|-|0|-|0|-|0       

0
Шаг 1:
    Сигнал:               Регистры:            Out: 
 0 0 0 0 0 0 0  1|-|0|-|0|-|0|-|0|-|0|-|0|-|0   1   

Шаг 2:
   Сигнал:              Регистры:            Out: 
 0 0 0 0 0 0  1|-|1|-|0|-|0|-|0|-|0|-|0|-|0  1 1  

Шаг 3:
  Сигнал:             Регистры:             Out: 
 0 0 0 0 0  1|-|1|-|1|-|0|-|0|-|0|-|0|-|0  1 1 1 

Шаг 4:
 Сигнал:            Регистры:              Out:  
 0 0 0 0  0|-|1|-|1|-|1|-|0|-|0|-|0|-|0  0 1 1 1 

Шаг 5:
 Сигнал:            Регистры:               Out:   
  0 0 0   1|-|0|-|1|-|1|-|1|-|0|-|0|-|0  1 0 1 1 1 

Шаг 6:
 Сигнал:            Регистры:                Out:    
   0 0    0|-|1|-|0|-|1|-|1|-|1|-|0|-|0  0 1 0 1 1 1 

Шаг 7:
 Сигнал:            Регистры:                 Out:     
    0     0|-|0|-|1|-|0|-|1|-|1|-|1|-|0  0 0 1 0 1 1 1 

Шаг 8:
 Сигнал:            Регистры:                  Out:      
          1|-

In [11]:
n = 8
q = 2
begin_state = [0] * n
numerator = [1, 0, 0, 0, 0, 0, 0, 1]
denominator = [1, 0, 1, 1]
msg = [1]

register_division(n, q, begin_state, numerator, denominator, msg, n - 1)

Начальное состояние:
     Сигнал:                Регистры:            Out: 
 0 0 0 0 0 0 0 1  0|-|0|-|0|-|0|-|0|-|0|-|0|-|0       

0
Шаг 1:
    Сигнал:               Регистры:            Out: 
 0 0 0 0 0 0 0  1|-|0|-|0|-|0|-|0|-|0|-|0|-|0   1   

Шаг 2:
   Сигнал:              Регистры:            Out: 
 0 0 0 0 0 0  0|-|1|-|0|-|0|-|0|-|0|-|0|-|0  0 1  

Шаг 3:
  Сигнал:             Регистры:             Out: 
 0 0 0 0 0  1|-|0|-|1|-|0|-|0|-|0|-|0|-|0  1 0 1 

Шаг 4:
 Сигнал:            Регистры:              Out:  
 0 0 0 0  1|-|1|-|0|-|1|-|0|-|0|-|0|-|0  1 1 0 1 

Шаг 5:
 Сигнал:            Регистры:               Out:   
  0 0 0   1|-|1|-|1|-|0|-|1|-|0|-|0|-|0  1 1 1 0 1 

Шаг 6:
 Сигнал:            Регистры:                Out:    
   0 0    0|-|1|-|1|-|1|-|0|-|1|-|0|-|0  0 1 1 1 0 1 

Шаг 7:
 Сигнал:            Регистры:                 Out:     
    0     0|-|0|-|1|-|1|-|1|-|0|-|1|-|0  0 0 1 1 1 0 1 

Шаг 8:
 Сигнал:            Регистры:                  Out:      
          1|-

In [12]:
n = 2
q = 2
begin_state = [0, 0]
numerator = [0, 1]
denominator = [0, 1]
msg = [1]

register_division(n, q, begin_state, numerator, denominator, msg, n - 1)

Начальное состояние:
 Сигнал:  Регистры:  Out: 
   0 1      0|-|0         

1
Шаг 1:
 Сигнал:  Регистры:  Out: 
    0       1|-|0         

Шаг 2:
 Сигнал:  Регистры:  Out: 
            0|-|1     1   

